# Fit and refine WCS

This script will try and fit a WCS to an exposure. Fiting a WCS consists of iteratively matching source and reference catalog sources where WCS is recalculated for each matching untill maximal number of iterations are achieved or untill maximal matching distance is smaller than set treshold. 

To perform WCS fitting we need to construct science source catalog and reference source catalog again. Because this exercise is getting a bit repetitive we will look into ways data can be persisted and loaded from disk without perfoming the Tasks again. 

First step is, again, to setup the Spark paths. This only needs to be run once per lifetime of a notebook.

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.setAppName("Something's happening.")
conf.setMaster("spark://ip-172-31-40-212.us-west-2.compute.internal:7077")
sc = SparkContext(conf=conf)
sc.addPyFile("/home/ec2-user/lsstspark.zip")

### Creating Science and Reference catalogs

For more details see DetectMeasureEstimate and Matching2RefCats notebooks. This is the same exact procedure.

In [2]:
import lsstspark as ls

# set up refObjLoader and refCat configurations
refCatConf = ls.DatasetConfig()
refCatConf.ref_dataset_name  = "ps1_pv3_3pi_20170110"
refCatConf.indexer = "HTM"
refCatLoc = "s3://dinolsstspark/small_ref_cats/"

# reference object setup
refObjConf = ls.LoadIndexedReferenceObjectsConfig()
refObjConf.filterMap = {'u': 'g', 'Y': 'y', "VR": "r"}
refObjConf.ref_dataset_name = refCatConf.ref_dataset_name

# register exposures we want to process
calexppath = "s3://dinolsstspark/sci_exposures/calexp-0308355_01.fits"
expPathsRDD = sc.parallelize([calexppath])

#ingest them as ExposureF objects
expRDD = expPathsRDD.map(ls.toExposureF)

#estimate PSF, detect and measure sources
estDetMeas = ls.detect_and_measure(psfIterations=2, doMeasurePsf=True)
estDetMeasRDD = expRDD.map(estDetMeas)

# we only need the source catalogs - or in this case a source catalog
scienceCatsRDD = estDetMeasRDD.map(lambda x: x[2])

# create the reference catalogs
refCatCreator = ls.create_reference_catalogs(refCatConf, refCatLoc, refObjConf)
referenceCatsRDD = expRDD.map(refCatCreator)

### Fitting WCS 

To fit a WCS we will be calling the `wcs_solver` wrapper function to set up and return a function `solver` that fits a new WCS. The packing order and logic are identical to that of Matching2RefCats notebook except the triplets that need to be passed into the WCS fitter require an exposure and look like: `([(scienceCat, referenceCat), exposure], [(), ], ....)`.

In [3]:
# bundle the data together
catPairsRDD = scienceCatsRDD.zip(referenceCatsRDD)
catPairsWithExpRDD = catPairsRDD.zip(expRDD)

We can run the solver on the catalog pair with exposures now.

In [4]:
solver = ls.wcs_solver(max_iter=3, minMatchDistanceArcSec=0.001)
wcsRDD = catPairsWithExpRDD.map(solver)

### Inspecting the data

We can inspect the returned object. I can't figure out how to plot it in matplotlib or otherwise verify it easily because they use ds9 and it's a more complex WCS than that Astropy uses.

In [5]:
a = wcsRDD.first()
a

## Persisting data to disk

Considering how each step is a building block for the next, persisting data to disk is a useful feature to explore in order to reduce processing times required during in interactive session. We will persist the source and reference catalogs and see how to organize processing by reading them from disk instead of creating them by mapping `detect_and_measure` and `create_reference_catalogs` to RDD of exposures.

Reference catalogs are instances of `SimpleCatalog` while science catalog of sources detected on the image are instances of `SourceCatalog`. Both can be persisted to disk by calling a method `.writeFits` on them. The `persistCatalog` function from the utils module is a light wrapper around this method.

The problem is apparent. Without the Butler and its policies we need a way to create file paths and file names such that no two are the same and preferably such that they remain in a form that is mapable to exposures that were used to create them. These names must be constructed in advance and then zipped with the source catalog RDD so that they match up. 

Unique exposure IDs can be retrieved by the `get_exposure_id` function and after zipping these IDs with source catalogs they can be persisted to disk by using the `persist_catalog` function. Both live in the utils module. 

In [6]:
exposureIdsRDD = expRDD.map(ls.utils.get_exposure_id)
sciCatsWithId  = scienceCatsRDD.zip(exposureIdsRDD)
refCatsWithId  = referenceCatsRDD.zip(exposureIdsRDD)

The `persist_catalog` function is a wrapper to a persisting function that enables file names and file paths to be set. File names are set by providing a string consistent with python string `.format` method (i.e. `descriptiveName_{0}`) where the curly braces will be replaced by whatever id is provided bundled with the catalog.

In [7]:
persist = ls.persist_catalog(location="s3://dinolsstspark/scicats", frmt="sciCat_{0}.fits")
sciCatLocsRDD = sciCatsWithId.map(persist)

persist = ls.persist_catalog(location="s3://dinolsstspark/refcats", frmt="refCat_{0}.fits")
refCatLocsRDD = refCatsWithId.map(persist)

In [8]:
print("Sci Cat locations: ", sciCatLocsRDD.collect())
print("Ref Cat locations: ", refCatLocsRDD.collect())

Sci Cat locations:  ['~/s3-drive/scicats/sciCat_30835501.fits']
Ref Cat locations:  ['~/s3-drive/refcats/refCat_30835501.fits']


### Reading data from disk

What remains is to see how to load that persisted dataset from disk and avoid creating them in the future. Unfortunately I could not come up with anything meaningful here except mapping the exact LSST datatypes `.readFits` methods to them. This is why Butler is good.

In [9]:
referenceCatsRDD = refCatLocsRDD.map(ls.utils.get_simple_catalog)
scienceCatsRDD   = sciCatLocsRDD.map(ls.utils.get_source_catalog)

In [10]:
referenceCatsRDD.first()

<class 'lsst.afw.table.simple.simple.SimpleCatalog'>
        id           coord_ra      coord_dec    ...   centroid_y   hasCentroid
                       rad            rad       ...                           
----------------- ------------- --------------- ... -------------- -----------
77931960590339852 3.42187543438 -0.437211352197 ...  1980.72917493        True
77931961667799703 3.42375616101 -0.437213817775 ...  3316.85976403        True
77931961833919988 3.42404509708 -0.437209744983 ...  3522.05435308        True
77931962033659504 3.42439471897  -0.43721671333 ...  3770.40555706        True
77941958772067733 3.41870249524 -0.437100500595 ... -274.511046018        True
77941958851788561 3.41884168367  -0.43708772983 ... -175.727643903        True
77941958880372033 3.41889069887 -0.437180284547 ...   -140.1856968        True
77941958891654634 3.41891037123 -0.437142515085 ... -126.509342847        True
77941958906214945 3.41893576869 -0.437137568583 ... -108.505243617        True

In [11]:
scienceCatsRDD.first()

<class 'lsst.afw.table.source.source.SourceCatalog'>
 id     coord_ra      coord_dec    ... calib_psfUsed calib_psf_reserved
          rad            rad       ...                                 
---- ------------- --------------- ... ------------- ------------------
   1 3.41911845014 -0.434563239167 ...         False              False
   2 3.41911594273 -0.435494980064 ...         False              False
   3 3.41913114669 -0.435591787724 ...         False              False
   4 3.41911046645 -0.435664214954 ...         False              False
   5 3.41911697767 -0.435167869754 ...         False              False
   6 3.41912025616  -0.43600637788 ...         False              False
   7 3.41912053567 -0.436575041883 ...         False              False
   8 3.41913114008 -0.435530673985 ...         False              False
   9 3.41912734772  -0.43604344957 ...         False              False
  10 3.41913992651 -0.435909475664 ...         False              False
 ...       